In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import opendatasets as od
import os
from zipfile import ZipFile

import plotly.express as px

%matplotlib inline

In [ ]:
walmart = pd.read_csv('./train.csv')
stores = pd.read_csv('./store.csv')
features = pd.read_csv('./features.csv')
# testing = pd.read_csv('./test.csv')

stores.info()

In [ ]:
features.drop(columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)

Removing our NULL/NA values from our dataframes prior to merging our 3 datasets into a single larger dataset

In [ ]:
merged = walmart.merge(stores, how='left').merge(features, how='left')
# testing_merged = testing.merge(stores, how='left').merge(features, how='left')

In [ ]:
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear'] = (df.Date.dt.isocalendar().week)*1.0   
    
split_date(merged) 

In [ ]:
merged.info()

Our first goal is to visualize the data that groups sales by the DMA. We will do so to create some baseline understanding of what each locations total sales look like.

In [ ]:
merged_dummy = pd.get_dummies(merged)

In [ ]:
dmacounts = merged.Store.value_counts().to_dict()
df = pd.DataFrame(list(dmacounts.items()), columns=['DMA', 'Store'])

# fig = px.pie(df, values='Store', names='DMA', title='Stores per DMA', labels = {'DMA':'DMA', 'Store':'Store'})

# fig = px.bar(df, x='DMA', y='Store', color='DMA', title='Store Types')

fig = px.pie(merged, values = 'Weekly_Sales', names = 'DMA' ,title = 'Weekly Sales per DMA', labels = {'DMA':'DMA', 'Weekly_Sales':'Weekly Sales'})

fig2 = px.bar(merged, x = 'DMA', y = 'Weekly_Sales', color = 'DMA', title = 'Weekly Sales per DMA')

# fig.show()
fig2.show()